# Scraping

This notebook follows an example of scraping Nadlan Gov. It yeilds a query csv in 'data/gov' folder which will be avilable of off line analysis. 

Follow the comments to modify and try different queries.

**Note:** This proceedure takes time, queries with a mutiple results could take a long time. Try not to query about more than one neigberhood.

In [20]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service

# import org.openqa.selenium.JavascriptExecutor as JavascriptExecutor
import pandas as pd

In [21]:
# update to your driver path
DRIVER_PATH = 'C:/Users/IMOE001/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe'

url = 'https://www.nadlan.gov.il/?view=neighborhood&id=65210960&page=deals'

service = Service(DRIVER_PATH)
browser = webdriver.Chrome(service=service)
browser.get(url)

# now wait for load
browser.implicitly_wait(5)

browser.set_window_size(1500, 1000)

In [22]:
# button = browser.find_element(By.CLASS_NAME, 'filterBtn')
# browser.execute_script("arguments[0].click();", button)
# time.sleep(0.5)  # Give time for the DOM to update

# button = browser.find_elements(By.CLASS_NAME, 'btn')[3]
# browser.execute_script("arguments[0].click();", button)
# time.sleep(0.5)  # Give time for the DOM to update

# button = browser.find_elements(By.CLASS_NAME, 'dropdownBtn')[13]
# browser.execute_script("arguments[0].click();", button)
# time.sleep(0.5)  # Give time for the DOM to update

# button = browser.find_element(By.CLASS_NAME, 'filterBtn')
# browser.execute_script("arguments[0].click();", button)
# time.sleep(0.5)  # Give time for the DOM to update

In [23]:
class infinite_scroll(object):
   
    def __init__(self, last):
    
        self.last = last

    def __call__(self, driver):
        new = driver.execute_script('return document.body.scrollHeight')  
        if new > self.last:
            return new
        else:
            return False

In [24]:
last_height = browser.execute_script('return document.body.scrollHeight')
print(last_height)
flag=1

while flag==1:
    browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')

    try:
        w = wait(browser, 10)

        new_height = w.until(infinite_scroll( last_height))
        last_height = new_height

    except:
        print("End of page reached")
        flag = 0

2647
End of page reached


In [25]:
relevant_col_names = ['קומות במבנה','שנת בנייה','תאריך עסקה', 'כתובת', 'גוש/חלקה/תת-חלקה', 'סוג נכס', 'חדרים', 'קומה', 'מ"ר','מחיר']
relevant_col_dict = {c:[] for c in relevant_col_names}

In [26]:
def safe_get(features, idx):
    return features[idx].text if len(features) > idx else ""

In [27]:
has_next = True

while has_next:
    table = browser.find_elements(By.CLASS_NAME, "mainTable")[0]
    sell_row_data = table.find_elements(By.CLASS_NAME, "mainTable__row")

    for i in range(1, len(sell_row_data)):
        ActionChains(browser).move_to_element(sell_row_data[i]).perform()

        main_features = sell_row_data[i].find_elements(By.CLASS_NAME,"mainTable__cell")
        relevant_col_dict['כתובת'].append(safe_get(main_features, 1))
        relevant_col_dict['מ"ר'].append(safe_get(main_features, 2))
        relevant_col_dict['תאריך עסקה'].append(safe_get(main_features, 3))
        relevant_col_dict['מחיר'].append(safe_get(main_features, 4))
        relevant_col_dict['גוש/חלקה/תת-חלקה'].append(safe_get(main_features, 5))
        relevant_col_dict['סוג נכס'].append(safe_get(main_features, 6))
        relevant_col_dict['חדרים'].append(safe_get(main_features, 7))
        relevant_col_dict['קומה'].append(safe_get(main_features, 8))

        # Click the collapse arrow using JS to avoid interception
        arrow = sell_row_data[i].find_elements(By.CLASS_NAME, "collapseArrow")[0]
        browser.execute_script("arguments[0].click();", arrow)
        time.sleep(0.5)  # Give time for the DOM to update

        sell_row_data_collapse = table.find_elements(By.CLASS_NAME, "innerTablesContainer")[0]
        additional_features = sell_row_data_collapse.find_elements(By.CLASS_NAME,"innerTable__cell")
        # get previous deals
        # print(safe_get(features, 8), safe_get(features, 9...)
        relevant_col_dict['שנת בנייה'].append(safe_get(additional_features, 3))
        relevant_col_dict['קומות במבנה'].append(safe_get(additional_features, 5))

        previous_deals_index = 8
        while safe_get(additional_features, previous_deals_index) != "":
            relevant_col_dict['כתובת'].append(safe_get(main_features, 1))
            relevant_col_dict['מ"ר'].append(safe_get(main_features, 2))
            relevant_col_dict['תאריך עסקה'].append(safe_get(additional_features, previous_deals_index))
            relevant_col_dict['מחיר'].append(safe_get(additional_features, previous_deals_index + 1))
            relevant_col_dict['גוש/חלקה/תת-חלקה'].append(safe_get(main_features, 5))
            relevant_col_dict['סוג נכס'].append(safe_get(main_features, 6))
            relevant_col_dict['חדרים'].append(safe_get(main_features, 7))
            relevant_col_dict['קומה'].append(safe_get(main_features, 8))
            relevant_col_dict['שנת בנייה'].append(safe_get(additional_features, 3))
            relevant_col_dict['קומות במבנה'].append(safe_get(additional_features, 5))
            previous_deals_index += 2

        browser.execute_script("arguments[0].click();", arrow)
        time.sleep(0.5)  # Give time for the DOM to update

    next_button = browser.find_elements(By.ID, "next")
    if len(next_button) == 0:
        has_next = False
        break
    ActionChains(browser).move_to_element(next_button[0]).perform()
    browser.execute_script("arguments[0].click();", next_button[0])
    time.sleep(1.5)  # Give time for the DOM to update

print(relevant_col_dict)

KeyboardInterrupt: 

In [28]:
nadlan_df = pd.DataFrame.from_dict(relevant_col_dict)

for k,v in relevant_col_dict.items():
    print(f'{k}: {len(v)}')

קומות במבנה: 6
שנת בנייה: 6
תאריך עסקה: 6
כתובת: 6
גוש/חלקה/תת-חלקה: 6
סוג נכס: 6
חדרים: 6
קומה: 6
מ"ר: 6
מחיר: 6


In [29]:
# name your csv
csv_name = 'Kiryat_Haim_East'

nadlan_df.to_csv(f'data/gov/{csv_name}.csv', index=False)